In [1]:
import warnings
warnings.filterwarnings('ignore')
import sys, glob
sys.path.append('/code')
from tools.docking.vina import vina_dock_multi
from tools.docking.preparation import define_vina_box
from example_notebooks.ui.file_widgets import PathSelector, DockButtons, InputBoxSet, log_progress
import os, shutil
from tqdm import tqdm

ffevaluate module is in beta version

Please cite HTMD: Doerr et al.(2016)JCTC,12,1845. 
https://dx.doi.org/10.1021/acs.jctc.6b00049
Documentation: http://software.acellera.com/

You are on the latest HTMD version (1.15.2).



# Docking (vina) multiple ligands in a box defined by a reference ligand
---

### 1. Selecting some inputs:

---

**First we need to select some inputs, namely:**
- the protein we want to dock into (pdb file)
- the reference ligand to define a box (mol/sdf)
- the ligands we want to dock (sdf)
- tan output directory

**To do this:**  
*(N.B. run the next 2 cells to show the file dialogues)*
1. click on the arrow in the dropdown box at the top
2. select the relevant file in the dialogue 
3. click the button next to either protein, query ligands or reference to set the file

*repeat this until all files are selected*

4. type the directory you want to get the results of the docking in into the 'Docking dir' input
5. select 'Set' to set the directory if the directory exists or create to create it, and then set to set it



In [2]:
f = PathSelector(start_dir=os.path.join(os.getcwd()))
display(f.accord)
b = DockButtons(f)
b.render_buttons()

A Jupyter Widget

A Jupyter Widget

In [3]:
i = InputBoxSet(tlabel='Docking dir:')
i.render()

A Jupyter Widget

/data/docking/XX02KALRNA-x1376_1
Created: /data/docking/XX02KALRNA-x1376_1/surprise_set
/data/docking/XX02KALRNA-x1376_1/surprise_set


---

### 2. Check the inputs 
---

Run the next cell, and check you are happy with the inputs and docking directory


In [8]:
pdb_file = b.pdb_button.description
ligands_file = b.lig_button.description
ref_file = b.ref_button.description
conf_file = 'conf.txt'
docking_directory = i.inbox.value

print(f'inputs: \n\t - pdb: {pdb_file} \n\t - ligs: {ligands_file} \
      \n\t - ref: {ref_file} \n\ndocking directory: {docking_directory}')

inputs: 
	 - pdb: /data/XX02KALRNA-x1376_1/XX02KALRNA-x1376_1.pdb 
	 - ligs: /data/XX02KALRNA-x1376_1/250_surprise_sucos.sdf       
	 - ref: /data/XX02KALRNA-x1376_1/XX02KALRNA-x1376_1.sdf 

docking directory: /data/docking/XX02KALRNA-x1376_1/surprise_set


---

### 3. Define a box from the reference ligand
---

Define a box around your reference ligand, and then write it out to a configuration file for autodock vina

There are some parameters that you can change here:

```
define_vina_box:
    Use a reference ligand to produce a box definition for autodock vina
    
        :param ref_lig: a mol or sdf file of the reference ligand
        :param output: the name of the output file
        :param bufx: distance (A) of buffer in x direction (default=10)
        :param bufy: distance (A) of buffer in y direction (default=10)
        :param bufz: distance (A) of buffer in z direction (default=10)
        :param exhaustiveness: exhaustivness of the vina calculation (default=20)
        :param seed: random seed for vina (default=None)
        :param cpu: number of cpu's to use in vina calculation
```

In [9]:
define_vina_box(ligands_file, conf_file)

---

### 4. Dock!
---

There are some parameters that you can change here:

```
vina_dock_multi:
    Dock multiple ligands from an sdf file against a protein, with a given configuration file with vina. 
    Also does preparation from pdb and sdf
    
        :param protein_pdb: the input pdb file to prepare and dock against
        :param ligands_sdf: an sdf file of multiple ligands to split, prepare and dock
        :param conf: the path of a configuration file (required for now)
        :param docking_dir: the output directory
        :param multiproc: use multi-processing? (boolean)
        :param exe: the path to the vina executable
```

But if you have followed the steps in the above boxes, you can just run this cell without a worry!


In [20]:
vina_dock_multi(protein_pdb=pdb_file, ligands_sdf=ligands_file, 
                conf=conf_file, docking_dir=docking_directory, multiproc=True)

docking_dir=docking_directory

if not os.path.isdir(os.path.join(docking_dir, 'docked/')):
    os.mkdir(os.path.join(docking_dir, 'docked/'))
if not os.path.isdir(os.path.join(docking_dir, 'logs/')):
    os.mkdir(os.path.join(docking_dir, 'logs/'))
if not os.path.isdir(os.path.join(docking_dir, 'inputs/')):
    os.mkdir(os.path.join(docking_dir, 'inputs/'))

docked_files = glob.glob(os.path.join(docking_dir, '*_docked.pdbqt'))
for f in docked_files:
    shutil.move(f, os.path.join(docking_dir, 'docked/' + f.split('/')[-1]))

logs = glob.glob(os.path.join(docking_dir, '*_docked.log'))
for f in logs:
    shutil.move(f, os.path.join(docking_dir, 'logs/' + f.split('/')[-1]))

in_ligs = glob.glob(os.path.join(docking_dir, '*_prepared.pdbqt'))
for f in in_ligs:
    shutil.move(f, os.path.join(docking_dir, 'inputs/' + f.split('/')[-1]))

print_str = f"Docking complete! Your results have been saved to: \
\n\t - docked ligands: {os.path.join(docking_directory, 'docked/')} \
\n\t - log files: {os.path.join(docking_directory, 'logs/')} \
\n\t - prepared input files: {os.path.join(docking_directory, 'inputs/')}"
        
print(print_str)

Docking complete! Your results have been saved to: 
	 - docked ligands: /data/docking/XX02KALRNA-x1376_1/surprise_set/docked/ 
	 - log files: /data/docking/XX02KALRNA-x1376_1/surprise_set/logs/ 
	 - prepared input files: /data/docking/XX02KALRNA-x1376_1/surprise_set/inputs/


In [33]:
def doit(glob_dir):
    file_names = glob.glob(glob_dir + '*.pdbqt')
    everything = []
    failures = []
    print (f'Found {len(file_names)} pdbqt files')
    for file_name in file_names:
        file = open(file_name)
        lines = file.readlines()
        file.close()
        try:
            line = lines[1]
            result = float(line.split(':')[1].split()[0])
            everything.append([result, file_name])
        except:
            failures.append(file_name)
            
    from operator import itemgetter
    
    ev = sorted(everything, key=itemgetter(0))
#     everything.sort(lambda x,y: cmp(x[0], y[0]))
#     part = everything[:n]
#     for p in part:
#         print(p[1])
#     if len(failures) > 0:
#         print(f'WARNING: {len(failures)} pdbqt files could not be processed')
    print(ev)

In [34]:
doit('/data/docking/XX02KALRNA-x1376_1/surprise_set/docked/')

Found 248 pdbqt files
[[-7.3, '/data/docking/XX02KALRNA-x1376_1/surprise_set/docked/250_surprise_sucos5_prepared_docked.pdbqt'], [-7.0, '/data/docking/XX02KALRNA-x1376_1/surprise_set/docked/250_surprise_sucos6_prepared_docked.pdbqt'], [-6.9, '/data/docking/XX02KALRNA-x1376_1/surprise_set/docked/250_surprise_sucos7_prepared_docked.pdbqt'], [-6.9, '/data/docking/XX02KALRNA-x1376_1/surprise_set/docked/250_surprise_sucos4_prepared_docked.pdbqt'], [-6.8, '/data/docking/XX02KALRNA-x1376_1/surprise_set/docked/250_surprise_sucos19_prepared_docked.pdbqt'], [-6.8, '/data/docking/XX02KALRNA-x1376_1/surprise_set/docked/250_surprise_sucos241_prepared_docked.pdbqt'], [-6.8, '/data/docking/XX02KALRNA-x1376_1/surprise_set/docked/250_surprise_sucos214_prepared_docked.pdbqt'], [-6.7, '/data/docking/XX02KALRNA-x1376_1/surprise_set/docked/250_surprise_sucos246_prepared_docked.pdbqt'], [-6.6, '/data/docking/XX02KALRNA-x1376_1/surprise_set/docked/250_surprise_sucos23_prepared_docked.pdbqt'], [-6.6, '/data/d